## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test, mean, std

## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
input_shape = (32,32,3)
#32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape, activation='relu'))  
classifier.add(BatchNormalization())

# '''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

# 卷積組合
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

# flatten
classifier.add(Flatten())

# FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

# 輸出
classifier.add(Dense(output_dim=10, activation='softmax'))

classifier.summary()
# 超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)             

In [5]:
print(y_train.shape)

(50000, 10)


In [7]:
classifier.fit(x_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
50000/50000 [==============================] - 104s 2ms/step - loss: 0.1343 - accuracy: 0.9538
Epoch 2/10
50000/50000 [==============================] - 105s 2ms/step - loss: 0.1224 - accuracy: 0.9577
Epoch 3/10
50000/50000 [==============================] - 104s 2ms/step - loss: 0.1021 - accuracy: 0.9656
Epoch 4/10
50000/50000 [==============================] - 100s 2ms/step - loss: 0.0906 - accuracy: 0.9694
Epoch 5/10
50000/50000 [==============================] - 97s 2ms/step - loss: 0.0840 - accuracy: 0.9716
Epoch 6/10
50000/50000 [==============================] - 110s 2ms/step - loss: 0.0827 - accuracy: 0.9710
Epoch 7/10
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0819 - accuracy: 0.9710
Epoch 8/10
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0758 - accuracy: 0.9734
Epoch 9/10
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0675 - accuracy: 0.9762
Epoch 10/10
50000/50000 [=========================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [8]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.0935236e-05, 6.7327200e-14, 2.0927180e-09, 9.7943211e-01,
        3.2133973e-06, 3.8395342e-10, 8.6132062e-13, 1.0132465e-10,
        2.0523591e-02, 9.2766810e-17]], dtype=float32)